# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re
import warnings
import pickle
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import jaccard_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///CleanedMessagesDB.db')
df = pd.read_sql("SELECT * FROM messages", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [4]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

In [5]:
def tokenize(text):
    #small casing and removing non-needed punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize
    tokens = word_tokenize(text)
    
    # lemmatize and stop words removal
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, shuffle = True, random_state=0)
x_train.shape, y_train.shape, Y.shape, X.shape

((18351,), (18351, 36), (26216, 36), (26216,))

In [8]:
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(x_test)

In [10]:
for i, column in enumerate(y_test):
    print(column)
    print(classification_report(y_test[column], y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.60      0.45      0.51      1787
          1       0.85      0.91      0.88      6028
          2       0.27      0.48      0.35        50

avg / total       0.79      0.80      0.79      7865

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      6481
          1       0.81      0.42      0.56      1384

avg / total       0.87      0.88      0.87      7865

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7838
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      7865

aid_related
             precision    recall  f1-score   support

          0       0.74      0.87      0.80      4572
          1       0.76      0.59      0.66      3293

avg / total       0.75      0.75      0.74      7865

medical_help
             precision    recall  f1-sco

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f2d3491a7b8>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,

In [12]:
pipeline.get_params().values()

dict_values([None, [('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f2d3491a7b8>, use_idf=True,
        vocabulary=None)), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1)

In [13]:
parameters = {
    'tfidf__max_df': (0.8, 1.0),
    'clf__estimator__n_estimators': [10, 50],
    'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1)

In [14]:
cv.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__max_df': (0.8, 1.0), 'clf__estimator__n_estimators': [10, 50], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
y_pred = cv.predict(x_test)

In [17]:
for i, column in enumerate(y_test):
    print(column)
    print(classification_report(y_test[column], y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.67      0.43      0.52      1787
          1       0.85      0.93      0.89      6028
          2       0.37      0.46      0.41        50

avg / total       0.80      0.82      0.80      7865

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      6481
          1       0.84      0.47      0.60      1384

avg / total       0.89      0.89      0.88      7865

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7838
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      7865

aid_related
             precision    recall  f1-score   support

          0       0.78      0.86      0.82      4572
          1       0.77      0.66      0.71      3293

avg / total       0.77      0.78      0.77      7865

medical_help
             precision    recall  f1-sco

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
from sklearn.decomposition import TruncatedSVD

In [20]:
pipeline_2 = Pipeline([
        ('vect' , CountVectorizer(tokenizer=tokenize)),
        ('tfidf' , TfidfTransformer()),
        ('svd',TruncatedSVD(n_components=100)),
        ('clf' , MultiOutputClassifier(RandomForestClassifier()))
])

In [21]:
pipeline_2.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [23]:
y_pred = pipeline_2.predict(x_test)

In [24]:
for i, column in enumerate(y_test):
    print(column)
    print(classification_report(y_test[column], y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.55      0.43      0.48      1787
          1       0.84      0.90      0.87      6028
          2       0.36      0.08      0.13        50

avg / total       0.77      0.78      0.77      7865

request
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      6481
          1       0.77      0.45      0.57      1384

avg / total       0.87      0.88      0.87      7865

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7838
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      7865

aid_related
             precision    recall  f1-score   support

          0       0.71      0.83      0.77      4572
          1       0.70      0.54      0.61      3293

avg / total       0.71      0.71      0.70      7865

medical_help
             precision    recall  f1-sco

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.